In [1]:
import pandas as pd
import numpy as np
from numpy import arctan as atan, sqrt, sin, cos, trapz
from scipy.stats import pearsonr

### Augmented measures

The original sensor data $a_f$, $a_l$, $a_v$ (front, vertical, and lateral sensors, respectively) are augmented messures as described in Wickramasinghe et al (2017, appendix B). Firstly angles $\theta$, $\alpha$, and $\beta$ are defined as follows:


$\theta \approx \tan^{-1}(a_f / \sqrt{a_v^2 + a_l ^2})$

$\alpha \approx \tan^{-1}(a_l / a_v)$

$\beta \approx \tan^{-1}(a_l / a_f)$


Then approximate acceleration signals $a_x$, $a_y$, and $a_z$ are derived as shown below:

$a_x \approx a_v\sin(\theta) + a_f\cos(\theta)$

$a_y \approx a_v\sin(\alpha) + a_l\cos(\alpha)$

$a_z \approx 1 + a_v\cos(\theta)\cos(\alpha) + a_l\sin(\alpha) + a_f\sin(\theta)$

### Features

Features are taken over a window $X_{[t_i - \delta t, t_i]}$:

| Feature | Description |
|:-:|:-|
| $a_f$ | Frontal acceleration |
| $a_l$ | Lateral acceleration |
| $a_v$ | Vertical acceleration |
| $\theta$ | Angle on Sagittal plane |
| $\alpha$ | Angle on Coronal plane |
| $\beta$ | Angle on Transverse plane |
| $a_x$ | Anteroposterior acceleration |
| $a_y$ | Mediolateral acceleration |
| $a_z$ | Dorsvental acceleration |
| $\mbox{mean}$$(a_f)$; $\mbox{mean}$$(a_l)$; $\mbox{mean}$$(a_v)$ |
| $\mbox{max}$$(a_f)$; $\mbox{max}$$(a_l)$; $\mbox{max}$$(a_v)$ |
| $\mbox{min}$$(a_f)$; $\mbox{min}$$(a_l)$; $\mbox{min}$$(a_v)$ |
| $\mbox{corr}$$(a_f\mbox{,}a_l)$; $\mbox{corr}$$(a_f\mbox{,}a_v)$; $\mbox{corr}$$(a_l\mbox{,}a_v)$ |
| $\mbox{mean}$ $(a_x)$; $\mbox{mean}$ $(a_y)$; $\mbox{mean}$ $(a_z)$ | 
| $\mbox{mean}$$(\theta)$; $\mbox{mean}$$(\alpha)$; $\mbox{mean}$$(\beta)$ |
| $\mbox{max}$$(\theta)$; $\mbox{max}$$(\alpha)$; $\mbox{max}$$(\beta)$ |
| $\mbox{min}$$(\theta)$; $\mbox{min}$$(\alpha)$; $\mbox{min}$$(\beta)$ |

In [2]:
from pathlib import Path

import pandas as pd

DATA_ROOT = Path('..') / 'data' 

dfs = []
activity_labels = ['bed', 'chair', 'lying', 'ambulating']
default_names = ['time', 'front', 'vertical', 'lateral', 'sensor_id', 'rssi', 'phase', 'frequency', 'activity']
for data_file in Path(DATA_ROOT).rglob('d[12]p??[FM]'):
    df = pd.read_csv(data_file, names=default_names)
    df['activity_label'] = df['activity'].apply(lambda i: activity_labels[i - 1])
    df['gender_label'] = str(data_file)[-1]
    df['participant'] = data_file.name
    
    # Add a column indicating order of the activities for a particiapnt.
    df = df.sort_values(by=['time'])
    df['activity_sequence'] = (df['activity'].shift(1) != df['activity']).cumsum()
    dfs.append(df)

sensor_df = pd.concat(dfs, axis='index')
sensor_df = sensor_df.sort_values(by=['participant', 'time'])
sensor_df['room'] = sensor_df['participant'].str.slice(1, 2).astype(int)

sensor_df.head()

,time,front,vertical,lateral,sensor_id,rssi,phase,frequency,activity,activity_label,gender_label,participant,activity_sequence,room
0,0.00,0.27203,1.00820,-0.082102,1,-63.5,2.4252,924.25,1,bed,M,d1p01M,1,1
1,0.50,0.27203,1.00820,-0.082102,1,-63.0,4.7369,921.75,1,bed,M,d1p01M,1,1
2,1.50,0.44791,0.91636,-0.013684,1,-63.5,3.0311,923.75,1,bed,M,d1p01M,1,1
3,1.75,0.44791,0.91636,-0.013684,1,-63.0,2.0371,921.25,1,bed,M,d1p01M,1,1
4,2.50,0.34238,0.96229,-0.059296,1,-63.5,5.8920,920.25,1,bed,M,d1p01M,1,1


Each row $i$ of data in the dataframe is treated as the trailing edge of a frame of data from $t_i-\delta t$ to $t_i$, within each frame.

In [18]:
def corr(x, y):
    if len(x) == len(y) == 1:
        return 0
    return pearsonr(x, y)

def get_frame(df, t, delta_t):
    frame_df = df[(df['time'] > t - delta_t) & (df['time'] <= t)]
    
    if not len(frame_df):
        return None

    t = frame_df['time']
    af = frame_df['front']
    al = frame_df['lateral']
    av = frame_df['vertical']

    theta = atan(af / sqrt(av**2 + al**2))
    alpha = atan(al / av)
    beta = atan(al / af)

    ax = av * sin(theta) + af * cos(theta)
    ay = av * sin(alpha) + al * cos(alpha)
    az = 1 + av * cos(theta) * cos(alpha) + al * sin(alpha) + af * sin(theta)
    
    
    vx = trapz(ax)
    vy = trapz(ay)
    vz = trapz(az)
        
    tmax_af = t[af.idxmax()]
    tmax_al = t[al.idxmax()]
    tmax_av = t[av.idxmax()]
    
    tmin_af = t[af.idxmin()]
    tmin_al = t[al.idxmin()]
    tmin_av = t[av.idxmin()]
    
    tmax_theta = t[theta.idxmax()]
    tmax_alpha = t[alpha.idxmax()]
    tmax_beta = t[beta.idxmax()]
    
    tmin_theta = t[theta.idxmin()]
    tmin_alpha = t[alpha.idxmin()]
    tmin_beta = t[beta.idxmin()]
    
    if af.nunique() > 1 and al.nunique() > 1 and av.nunique() > 1:
        coor_af_al = corr(af, al)
        coor_af_av = corr(af, av)
        coor_al_av = corr(al, av)
    else:
        coor_af_al = 0
        coor_af_av = 0
        coor_al_av = 0
        
    frame = dict({
        'class': frame_df['activity'].values[-1],
        't': t.values[-1],
        'af': af.values[-1],
        'al': al.values[-1],
        'av': av.values[-1],
        'ar': np.sqrt(af.values[-1] ** 2 + al.values[-1] ** 2 + av.values[-1] ** 2),
        'ax': ax.values[-1],
        'ay': ay.values[-1],
        'az': az.values[-1],
        'af_mean': af.mean(),
        'al_mean': al.mean(),
        'av_mean': av.mean(),
        'af_max': af.max(),
        'al_max': al.max(),
        'av_max': av.max(),
        'af_min': af.min(),
        'al_min': al.min(),
        'av_min': av.min(),
        'coor_af-al': coor_af_al,
        'coor_af-av': coor_af_av,
        'coor_al-av': coor_al_av,
        'ax_mean': ax.mean(),
        'ay_mean': ay.mean(),
        'az_mean': az.mean(),
        'tmax_af<tmin_af': tmax_af < tmin_af,
        'tmax_al<tmin_al': tmax_al < tmin_al,
        'tmax_av<tmin_av': tmax_av < tmin_av,
        'theta': theta.values[-1],
        'alpha': alpha.values[-1],
        'beta': beta.values[-1],
        'theta_mean': theta.mean(),
        'alpha_mean': alpha.mean(),
        'beta_mean': beta.mean(),
        'theta_max': theta.max(),
        'alpha_max': alpha.max(),
        'beta_max': beta.max(),
        'theta_min': theta.min(),
        'alpha_min': alpha.min(),
        'beta_min': beta.min(),
        'tmax_theta<tmin_theta': tmax_theta < tmin_theta,
        'tmax_alpha<tmin_alpha': tmax_alpha < tmin_alpha,
        'tmax_beta<tmin_beta': tmax_beta < tmin_beta,
        'vx': vx,
        'vy': vy,
        'vz': vz,
        'vr': np.sqrt(vx ** 2 + vy ** 2 + vz ** 2),
    })
    return frame

def generate_data(df, delta_t=2.0, t_freq=0.5):
    participants = df['participant'].unique()
    frames = []
    for participant in participants:
        print(participant)
        participant_df = df[df['participant'] == participant]
        min_t, max_t = participant_df['time'].min(), participant_df['time'].max()
        t = min_t
        while t <= max_t:
            frame = get_frame(participant_df, t, delta_t)
            frames.append(frame)
            t += t_freq
    return pd.DataFrame(frames)

In [19]:
rooms = sensor_df['room'].unique()
room = rooms[0]

room_df = sensor_df[sensor_df['room'] == room]


In [20]:
data_df = generate_data(room_df)

d1p01M
0    0.27203
Name: front, dtype: float64 0   -0.082102
Name: lateral, dtype: float64 0    1.0082
Name: vertical, dtype: float64
0    0.27203
1    0.27203
Name: front, dtype: float64 0   -0.082102
1   -0.082102
Name: lateral, dtype: float64 0    1.0082
1    1.0082
Name: vertical, dtype: float64
0    0.27203
1    0.27203
Name: front, dtype: float64 0   -0.082102
1   -0.082102
Name: lateral, dtype: float64 0    1.0082
1    1.0082
Name: vertical, dtype: float64
0    0.27203
1    0.27203
2    0.44791
Name: front, dtype: float64 0   -0.082102
1   -0.082102
2   -0.013684
Name: lateral, dtype: float64 0    1.00820
1    1.00820
2    0.91636
Name: vertical, dtype: float64
1    0.27203
2    0.44791
3    0.44791
Name: front, dtype: float64 1   -0.082102
2   -0.013684
3   -0.013684
Name: lateral, dtype: float64 1    1.00820
2    0.91636
3    0.91636
Name: vertical, dtype: float64
2    0.44791
3    0.44791
4    0.34238
Name: front, dtype: float64 2   -0.013684
3   -0.013684
4   -0.059296
Name

Name: front, dtype: float64 31    0.020525
Name: lateral, dtype: float64 31    0.92784
Name: vertical, dtype: float64
31    0.47136
Name: front, dtype: float64 31    0.020525
Name: lateral, dtype: float64 31    0.92784
Name: vertical, dtype: float64
32    0.2134
33    0.2134
Name: front, dtype: float64 32   -0.12771
33   -0.12771
Name: lateral, dtype: float64 32    1.0312
33    1.0312
Name: vertical, dtype: float64
32    0.2134
33    0.2134
34    0.2134
Name: front, dtype: float64 32   -0.12771
33   -0.12771
34   -0.12771
Name: lateral, dtype: float64 32    1.0312
33    1.0312
34    1.0312
Name: vertical, dtype: float64
32    0.2134
33    0.2134
34    0.2134
Name: front, dtype: float64 32   -0.12771
33   -0.12771
34   -0.12771
Name: lateral, dtype: float64 32    1.0312
33    1.0312
34    1.0312
Name: vertical, dtype: float64
32    0.21340
33    0.21340
34    0.21340
35    0.23685
Name: front, dtype: float64 32   -0.12771
33   -0.12771
34   -0.12771
35   -0.10491
Name: lateral, dtype: f

Name: front, dtype: float64 84   -0.013684
85   -0.150520
86   -0.150520
87   -0.116310
88   -0.116310
89   -0.116310
90   -0.173330
Name: lateral, dtype: float64 84   -0.36976
85   -0.18603
86   -0.18603
87   -0.23196
88   -0.23196
89   -0.23196
90   -0.19751
Name: vertical, dtype: float64
86    0.92865
87    1.03420
88    1.03420
89    1.03420
90    0.90520
91    1.15140
Name: front, dtype: float64 86   -0.150520
87   -0.116310
88   -0.116310
89   -0.116310
90   -0.173330
91   -0.047893
Name: lateral, dtype: float64 86   -0.18603
87   -0.23196
88   -0.23196
89   -0.23196
90   -0.19751
91   -0.30086
Name: vertical, dtype: float64
87    1.0342
88    1.0342
89    1.0342
90    0.9052
91    1.1514
92    0.9052
93    1.3039
Name: front, dtype: float64 87   -0.116310
88   -0.116310
89   -0.116310
90   -0.173330
91   -0.047893
92   -0.173330
93   -0.002281
Name: lateral, dtype: float64 87   -0.23196
88   -0.23196
89   -0.23196
90   -0.19751
91   -0.30086
92   -0.17454
93   -0.33531
Name: ver

Name: front, dtype: float64 146   -0.16192
147   -0.17333
148   -0.17333
149   -0.17333
150   -0.16192
Name: lateral, dtype: float64 146   -0.17454
147   -0.18603
148   -0.18603
149   -0.18603
150   -0.17454
Name: vertical, dtype: float64
147    0.91693
148    0.91693
149    0.91693
150    0.89348
151    0.91693
152    0.88175
153    0.88175
154    1.01070
Name: front, dtype: float64 147   -0.17333
148   -0.17333
149   -0.17333
150   -0.16192
151   -0.15052
152   -0.16192
153   -0.16192
154   -0.11631
Name: lateral, dtype: float64 147   -0.18603
148   -0.18603
149   -0.18603
150   -0.17454
151   -0.17454
152   -0.17454
153   -0.17454
154   -0.19751
Name: vertical, dtype: float64
148    0.91693
149    0.91693
150    0.89348
151    0.91693
152    0.88175
153    0.88175
154    1.01070
155    1.01070
Name: front, dtype: float64 148   -0.17333
149   -0.17333
150   -0.16192
151   -0.15052
152   -0.16192
153   -0.16192
154   -0.11631
155   -0.11631
Name: lateral, dtype: float64 148   -0.18603

Name: front, dtype: float64 191   -0.059296
192   -0.059296
193   -0.059296
Name: lateral, dtype: float64 191    0.97377
192    0.97377
193    0.97377
Name: vertical, dtype: float64
192    0.37756
193    0.37756
194    0.31893
195    0.31893
Name: front, dtype: float64 192   -0.059296
193   -0.059296
194   -0.082102
195   -0.082102
Name: lateral, dtype: float64 192    0.97377
193    0.97377
194    0.97377
195    0.97377
Name: vertical, dtype: float64
193    0.37756
194    0.31893
195    0.31893
196    0.31893
Name: front, dtype: float64 193   -0.059296
194   -0.082102
195   -0.082102
196   -0.082102
Name: lateral, dtype: float64 193    0.97377
194    0.97377
195    0.97377
196    0.97377
Name: vertical, dtype: float64
194    0.31893
195    0.31893
196    0.31893
197    0.31893
Name: front, dtype: float64 194   -0.082102
195   -0.082102
196   -0.082102
197   -0.082102
Name: lateral, dtype: float64 194    0.97377
195    0.97377
196    0.97377
197    0.97377
Name: vertical, dtype: float64

...

Name: front, dtype: float64 124    0.009122
Name: lateral, dtype: float64 124    0.70966
Name: vertical, dtype: float64
125    0.7176
126    0.7176
Name: front, dtype: float64 125   -0.093505
126   -0.093505
Name: lateral, dtype: float64 125    0.79004
126    0.79004
Name: vertical, dtype: float64
125    0.7176
126    0.7176
127    0.7176
Name: front, dtype: float64 125   -0.093505
126   -0.093505
127   -0.093505
Name: lateral, dtype: float64 125    0.79004
126    0.79004
127    0.79004
Name: vertical, dtype: float64
125    0.7176
126    0.7176
127    0.7176
128    0.7176
Name: front, dtype: float64 125   -0.093505
126   -0.093505
127   -0.093505
128   -0.093505
Name: lateral, dtype: float64 125    0.79004
126    0.79004
127    0.79004
128    0.79004
Name: vertical, dtype: float64
125    0.7176
126    0.7176
127    0.7176
128    0.7176
Name: front, dtype: float64 125   -0.093505
126   -0.093505
127   -0.093505
128   -0.093505
Name: lateral, dtype: float64 125    0.79004
126    0.79004


Name: vertical, dtype: float64
154   -0.079733
155   -0.079733
156    0.870030
157    0.870030
Name: front, dtype: float64 154    0.14596
155    0.14596
156    0.25999
157    0.25999
Name: lateral, dtype: float64 154    0.87042
155    0.87042
156    0.44555
157    0.44555
Name: vertical, dtype: float64
155   -0.079733
156    0.870030
157    0.870030
Name: front, dtype: float64 155    0.14596
156    0.25999
157    0.25999
Name: lateral, dtype: float64 155    0.87042
156    0.44555
157    0.44555
Name: vertical, dtype: float64
156    0.87003
157    0.87003
158    1.09280
Name: front, dtype: float64 156    0.25999
157    0.25999
158   -0.03649
Name: lateral, dtype: float64 156    0.44555
157    0.44555
158   -0.22048
Name: vertical, dtype: float64
157    0.87003
158    1.09280
159    1.05760
160    1.05760
Name: front, dtype: float64 157    0.25999
158   -0.03649
159   -0.13912
160   -0.13912
Name: lateral, dtype: float64 157    0.44555
158   -0.22048
159   -0.19751
160   -0.19751
Name: v

Name: vertical, dtype: float64
213    0.98728
214    0.95210
215    0.95210
216    0.95210
Name: front, dtype: float64 213   -0.24174
214   -0.24174
215   -0.24174
216   -0.24174
Name: lateral, dtype: float64 213   -0.17454
214   -0.15158
215   -0.15158
216   -0.15158
Name: vertical, dtype: float64
214    0.9521
215    0.9521
216    0.9521
217    1.2101
Name: front, dtype: float64 214   -0.24174
215   -0.24174
216   -0.24174
217   -0.13912
Name: lateral, dtype: float64 214   -0.15158
215   -0.15158
216   -0.15158
217   -0.31234
Name: vertical, dtype: float64
214    0.9521
215    0.9521
216    0.9521
217    1.2101
218    0.9521
Name: front, dtype: float64 214   -0.24174
215   -0.24174
216   -0.24174
217   -0.13912
218   -0.25315
Name: lateral, dtype: float64 214   -0.15158
215   -0.15158
216   -0.15158
217   -0.31234
218   -0.17454
Name: vertical, dtype: float64
216    0.9521
217    1.2101
218    0.9521
Name: front, dtype: float64 216   -0.24174
217   -0.13912
218   -0.25315
Name: later

Name: vertical, dtype: float64
258    0.92865
259    1.10450
260    1.01070
Name: front, dtype: float64 258   -0.27595
259   -0.19613
260   -0.24174
Name: lateral, dtype: float64 258   -0.17454
259   -0.25493
260   -0.19751
Name: vertical, dtype: float64
259    1.1045
260    1.0107
261    1.2218
Name: front, dtype: float64 259   -0.19613
260   -0.24174
261   -0.13912
Name: lateral, dtype: float64 259   -0.25493
260   -0.19751
261   -0.31234
Name: vertical, dtype: float64
260    1.0107
261    1.2218
Name: front, dtype: float64 260   -0.24174
261   -0.13912
Name: lateral, dtype: float64 260   -0.19751
261   -0.31234
Name: vertical, dtype: float64
260    1.0107
261    1.2218
Name: front, dtype: float64 260   -0.24174
261   -0.13912
Name: lateral, dtype: float64 260   -0.19751
261   -0.31234
Name: vertical, dtype: float64
261    1.2218
Name: front, dtype: float64 261   -0.13912
Name: lateral, dtype: float64 261   -0.31234
Name: vertical, dtype: float64
262    0.88175
263    0.90520
Name: f

KeyboardInterrupt: 

In [15]:
data_df.head()

""
